<a href="https://colab.research.google.com/github/rebeccahurwitz/bootswatch/blob/master/BST221_Problem_Set_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> **BST 221 Problem Set 6** </center>

### <center> Rebecca Hurwitz <br> 2023-04-14 </center> <br> 

### 1. We consider the LU algorithm. Implement this algorithm again or modify the implementation from the labs such that row pivoting is used. Use the ”maximum column pivot” stategy from the lectures. Test your implementation on the system $Ax = b$ with
$\mathbf{\mathit{A}}$ = $\mathbf{\begin{bmatrix} 0 & 3 & -2\\ 4 & -2 & 1\\ 2 & -1 & 1 \end{bmatrix}}$,   $\mathbf{\mathit{b}}$ = $\mathbf{\begin{bmatrix} -1 \\ -1 \\ 8 \end{bmatrix}}$  
### which was used to demonstrate the LU algorithm in class (since the slides did not use the maximum column pivot strategy, it is fine if the order in which your code permutes rows is not the same as done in the slides). Print out or return the matrices $L, U,$ and $P$. Implement forward and backward substitution to solve $Ax = b$ for $x$.

In [1]:
import numpy as np

class SystemSolver():

    def __init__(self,data = []):
        self.data = data

    def LUFactorization(self, A):
        nrows = A.shape[0]
        ncols = A.shape[1]
        n = nrows

        if nrows == ncols:
            L = np.identity(n)
            U = A.astype(float)
            P = np.identity(n) # Initialize the permutation matrix

            for i in range(n-1):
                # Find the maximum element in the current column below the diagonal
                max_index = np.argmax(np.abs(U[i:, i])) + i

                # Swap rows in U and P
                U[[i, max_index], i:n] = U[[max_index, i], i:n]
                P[[i, max_index], :] = P[[max_index, i], :]

                # Update L with the same row swaps as U
                if i > 0:
                    L[[i, max_index], :i] = L[[max_index, i], :i]

                for j in range(i+1, n):
                    L[j, i] = U[j, i]/U[i, i]
                    U[j, i:n] = U[j, i:n] - L[j, i]*U[i, i:n]

            return(P, L, U)

    def systemSolve(self, A, b):
        P, L, U = self.LUFactorization(A)

        # Apply permutation to the right-hand side b
        b = np.dot(P, b)

        n = len(b)
        y = [None]*n

        for j in range(n):
            y[j] = b[j]/L[j, j]
            for i in range(j+1, n):
                b[i] = b[i] - L[i, j]*y[j]

        x = [None]*n

        for j in range(n-1, -1, -1):
            x[j] = y[j]/U[j, j]
            for i in range(j):
                y[i] = y[i] - U[i, j]*x[j]

        return x


[1.0, 11.0, 17.0]

In [ ]:
#Check that this class works. Corresponds with example from lab slides.
SS = SystemSolver()
A = np.array([[0, 3, -2], [4, -2, 1], [2, -1, 1]]) 
b = [-1, -1, 8]
SS.systemSolve(A,b)

### 2. Implement an algorithm that solves $Ax = b$ using the Cholesky decomposition. Before running Cholesky, your code should check that $A$ is symmetric and positive definite (note that this implies $A$ must be square as well). Test your implementation on the system  
$\mathbf{\mathit{A}}$ = $\mathbf{\begin{bmatrix} 4 & 12 & -16\\ 12 & 37 & -43\\ -16 & -43 & 98 \end{bmatrix}}$,  $\mathbf{\mathit{b}}$ = $\mathbf{\begin{bmatrix} 0 \\ 6 \\ 39 \end{bmatrix}}$   
### Print out or return the matrix L. Use the foward and backward substitution algorithms from the previous question to calculate the solution vector x. Recall that no pivoting is required for the Cholesky decomposition.  

In [ ]:
def givens_rotation(A, decimals=10):
    """Perform QR decomposition of matrix A using Givens rotation."""
    (num_rows, num_cols) = np.shape(A)

    #Initialize orthogonal matrix Q and upper triangular matrix R.
    Q = np.identity(num_rows)
    R = np.copy(A)

    #Iterate over lower triangular matrix.
    #Iterate over columns left to right.
    for col in range(num_cols):
        #Iterate over rows bottom towards diagonal.
        for row in range(num_rows-1, col, -1):

            #Compute Givens rotation matrix and
            #zero-out lower triangular matrix entries.
            if R[row, col] != 0:
                a = R[col, col]
                b = R[row, col]
                r = np.sqrt(a**2 + b**2)
                c = a/r
                s = -b/r
                
                #Fill out the Givens rotation matrix
                G = np.identity(num_rows)
                G[col, col] = c
                G[row, row] = c
                G[row, col] = s
                G[col, row] = -s
                
                #Keep track of R and Q
                #Since G is orthogonal, G^{-1}=G^T
                R = np.dot(G, R)
                Q = np.dot(Q, G.T)

    #Return results, rounding as desired.
    return (Q.round(decimals), R.round(decimals))